<a href="https://colab.research.google.com/github/CyOwl-V/IDS-Project/blob/main/zeek%2BIsolationForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
cd'/content/gdrive/MyDrive/Colab Notebooks/zeek'

/content/gdrive/MyDrive/Colab Notebooks/zeek


In [7]:

# ایمپورت pandas
import pandas as pd

# تابع برای خواندن فایل Zeek conn.log
def parse_connlog(path):
    with open(path) as f:
        lines = f.readlines()

    # پیدا کردن خطی که اسامی ستون‌ها را دارد
    cols = []
    for line in lines:
        if line.startswith("#fields"):
            cols = line.strip().split("\t")[1:]  # حذف "#fields"
            break

    # خواندن داده‌های بدون خط‌های کامنت (#)
    data = [line.strip().split("\t") for line in lines if not line.startswith("#")]

    # ساخت DataFrame
    df = pd.DataFrame(data, columns=cols)
    return df

# خواندن دو فایل conn.log
df_normal = parse_connlog("conn-normal.log")
df_ddos = parse_connlog("conn-ddos.log")



In [8]:
# نمایش ۵ سطر اول هر فایل
print("✅ نمونه‌ای از ترافیک نرمال:")
print(df_normal.head())

print("\n✅ نمونه‌ای از ترافیک DDoS:")
print(df_ddos.head())


✅ نمونه‌ای از ترافیک نرمال:
                  ts                 uid        id.orig_h id.orig_p  \
0  1746614020.759923  Cvfap41RFDaLCazYn4  192.168.159.128     44300   
1  1746614027.671921  CF15Zg2KEbRJ9UIxhl  192.168.159.128     52416   
2  1746614027.672127  CPZ0Ad48AXRuMkxR42  192.168.159.128     58392   
3  1746614032.677563  CbYx0c4ygQHUo9rCh2  192.168.159.128     48564   
4  1746614037.784046  CoqgMI1MMzDNN2V18i  192.168.159.128     48564   

       id.resp_h id.resp_p proto service  duration orig_bytes  ... local_orig  \
0  192.168.159.1        53   tcp       -  4.096057          0  ...          T   
1  192.168.159.1        53   udp     dns         -          -  ...          T   
2  192.168.159.1        53   udp     dns         -          -  ...          T   
3  192.168.159.1        53   tcp       -  4.082357          0  ...          T   
4  192.168.159.1        53   tcp       -  2.048237          0  ...          T   

  local_resp missed_bytes history orig_pkts orig_ip_bytes 

In [11]:
# پیش‌پردازش ویژگی‌ها و تبدیل به عدد
def preprocess(df):
    return df[["duration", "orig_bytes", "resp_bytes", "orig_pkts", "resp_pkts"]].apply(pd.to_numeric, errors="coerce").fillna(0)

# اعمال پیش‌پردازش روی داده‌های نرمال و ddos
X_normal = preprocess(df_normal)
X_ddos = preprocess(df_ddos)

# افزودن برچسب
X_normal["label"] = 0  # ترافیک نرمال
X_ddos["label"] = 1    # ترافیک حمله

# ترکیب داده‌ها
data = pd.concat([X_normal, X_ddos], ignore_index=True)

# جدا کردن ویژگی‌ها و برچسب
X = data.drop("label", axis=1)
y = data["label"]


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# فرض: X = ویژگی‌ها، y = برچسب‌ها (label)

# تقسیم داده به آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# آموزش مدل Random Forest
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# پیش‌بینی روی داده تست
y_pred = clf.predict(X_test)

# نمایش گزارش عملکرد
print(" گزارش عملکرد مدل:")
print(classification_report(y_test, y_pred))


 گزارش عملکرد مدل:
              precision    recall  f1-score   support

           0       0.93      0.19      0.31       221
           1       1.00      1.00      1.00    172751

    accuracy                           1.00    172972
   macro avg       0.97      0.59      0.65    172972
weighted avg       1.00      1.00      1.00    172972

